In [1]:
import numpy as np 
import tensorflow as tf
%matplotlib inline 
from keras.models import Sequential
import matplotlib
import matplotlib.pyplot as plt


BATCH_SIZE = 10000
EPOCHS = 10
IMG_SIZE = 64

model_name = "pets_{}pixels_{}epochs".format(IMG_SIZE, EPOCHS)
model_name = 'pets_64pixels_15epochs'
tf.__version__

Using TensorFlow backend.


'1.12.0'

In [2]:
from keras.models import model_from_json
# load json and create model
json_file = open(model_name + '.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights(model_name + ".h5")
print("Loaded model from disk")

Loaded model from disk


In [3]:
import glob

TEST_IMAGES_PATH = 'C:/Users/madad/Documents/dataset/images/test-resized/'

# get the filepaths for the images
data_files = glob.glob(TEST_IMAGES_PATH + '**/*.jpg', recursive=True)

In [6]:
def predict_img(img_name):
    image = Image.open(img_name).convert('RGB')
    
    image = np.asarray(image)
    image.shape = (1, 64, 64, 3)
                
    prediction = np.argmax(model.predict(image))
    return prediction

In [8]:
import numpy as np
from PIL import Image
import os
import re

first_file = os.path.basename(data_files[0]).split('-')[0]
sum_pred = predict_img(data_files[0])
size_pred = 1
# result arrays
pet_id = []
pred_fin = []
# result dictionary
dict_res = {}

# loop over the images
# open and predict for each of them
for i in range (1, len(data_files)):   
    second_file = os.path.basename(data_files[i]).split('-')[0]
    
    prediction = predict_img(data_files[i]) 
    
    if first_file == second_file:
        sum_pred = sum_pred + prediction
        size_pred = size_pred + 1
        dict_res[first_file] = sum_pred
        # process the last pet_id 
        if i == len(data_files) - 1:
            pet_id.append(first_file)
            pred_fin.append(sum_pred//size_pred)
            dict_res[first_file] = sum_pred//size_pred
    else:
        pet_id.append(first_file)
        pred_fin.append(sum_pred//size_pred)
        dict_res[first_file] = sum_pred//size_pred
        sum_pred = prediction
        size_pred = 1
    first_file = second_file

assert(len(pet_id) == len(pred_fin))
assert(len(pet_id) == len(dict_res))

In [9]:
import pandas as pd

CSV_PATH = 'C:/Users/madad/Documents/dataset/sample_submission.csv'
empty_sub_df = pd.read_csv(CSV_PATH, header=0, usecols=['AdoptionSpeed', 'PetID'])

empty_sub_df.head()

,PetID,AdoptionSpeed
0,378fcc4fc,0
1,73c10e136,0
2,72000c4c5,0
3,e147a4b9f,0
4,43fbba852,0


In [10]:
def exists_imgs_for_pet(res_dict, petId):
    if petId in res_dict.keys():
        return 1
    return 0

In [11]:
sub_df = pd.DataFrame() 

for index, row in empty_sub_df.iterrows():

    petId = str(row['PetID'])
    
    sum_avg = 0
    size = 0 
    found = 0

    found = exists_imgs_for_pet(dict_res, petId)
    
    if found == 1:
        row['PetID'] = petId
        row['AdoptionSpeed'] = dict_res[petId]
    else:
        row['PetID'] = petId
        row['AdoptionSpeed'] = 4
    
    sub_df = sub_df.append([row],ignore_index=True)
        
sub_df.head()

,PetID,AdoptionSpeed
0,378fcc4fc,3
1,73c10e136,2
2,72000c4c5,3
3,e147a4b9f,4
4,43fbba852,4


In [12]:
sub_df.to_csv('submission.csv', encoding='utf-8', index=False)